# <span style="color:blue">Sample Python Client</span>

**This notebook contains sample Python code that demonstrates how to interface with the AccountLedger service for a particular usecase. The code mimics how a developer or product engineer may program business logic using the API.**

**Let's say we want to track loan payments and interest/late fees for a particular customer, Bob Bobberman. First, we would create a customer account for Bob with a unique customer account ID.**

**We can query the service to check a list of existing account IDs to make sure we don't choose a duplicate ID for Bob's account.**

In [13]:
import requests

# The AccountLedger Service is publicly hosted on an AWS EC2 instance
PUBLIC_IP = 'http://18.212.100.191'

r = requests.get(PUBLIC_IP + '/getCustomerAccountIds')
r.json()

[{'customerAccountId': 'abcd1234'},
 {'customerAccountId': '12345'},
 {'customerAccountId': 'abc12345'}]

**Now, let's <span style="color:red">create Bob's account</span>!**

In [10]:
import pprint
r = requests.post(PUBLIC_IP + '/createCustomerAccount', data = {
    'customerAccountId': 'bob123',
    'firstName': 'Bob',
    'lastName': 'Bobberman'
})

pprint.pprint(r.json())

{'_id': '5ed44fd829979d0e93750d68',
 'customerAccountId': 'bob123',
 'firstName': 'Bob',
 'lastName': 'Bobberman',
 'ledgers': {'applicationFee': [{'balance': 0,
                                 'timestamp': '1970-01-01T00:00:00.000Z'}],
             'interestFee': [{'balance': 0,
                              'timestamp': '1970-01-01T00:00:00.000Z'}],
             'interestFeePaid': [{'balance': 0,
                                  'timestamp': '1970-01-01T00:00:00.000Z'}],
             'lateFees': [{'balance': 0,
                           'timestamp': '1970-01-01T00:00:00.000Z'}],
             'principal': [{'balance': 0,
                            'timestamp': '1970-01-01T00:00:00.000Z'}],
             'principalPaid': [{'balance': 0,
                                'timestamp': '1970-01-01T00:00:00.000Z'}]}}


In [11]:
r = requests.get(PUBLIC_IP + '/getCustomerAccountIds')
r.json()

[{'customerAccountId': 'abcd1234'},
 {'customerAccountId': '12345'},
 {'customerAccountId': 'abc12345'},
 {'customerAccountId': 'bob123'}]

**Great! We've successfully created Bob's customer account. Upon creation, the AccountLedger service populates Bob's account record with default ledger balances and timestamps.**

**Next, let's say Bob takes out a loan on 1/1/2020, with principal amount of \\$1000, and an application fee amount of \\$10. Let's <span style="color:red">update the appropriate ledgers</span> for Bob's account.**

In [14]:
requests.post(PUBLIC_IP + '/updateLedgerBalance', data = {
    'customerAccountId': 'bob123',
    'ledgerName': 'principal',
    'newBalance': 1000,
    'timestamp': '2020-01-01'
})

requests.post(PUBLIC_IP + '/updateLedgerBalance', data = {
    'customerAccountId': 'bob123',
    'ledgerName': 'applicationFee',
    'newBalance': 10,
    'timestamp': '2020-01-01'
})

<Response [200]>

**For simplicity's sake, the AccountLedger service does not allow handling multiple balance changes at once but rather one after another. <span style="color:red">As long as the timstamp remains the same, these update events will correspond to the same date for the given customer account.</span>**

**Now that we've updated the principal and application fee, let's display Bob's <span style="color:red">most recent balance</span> for all the available ledger types.**

In [16]:
def print_current_balances():
    r = requests.get(PUBLIC_IP + '/getCurrentLedgerBalances?customerAccountId=bob123')
    pprint.pprint(r.json())
    
print_current_balances()

{'applicationFee': 10,
 'interestFee': 0,
 'interestFeePaid': 0,
 'lateFees': 0,
 'principal': 1000,
 'principalPaid': 0}


**We can also compare the current balance of specific ledger types with their <span style="color:red">PREVIOUS balance</span>.**

In [17]:
r = requests.get(PUBLIC_IP + '/getPreviousLedgerBalance?customerAccountId=bob123&ledgerName=principal')
pprint.pprint(r.json())

{'principal': 0}


**On 2/1/2020, an interest fee amount of \\$5 is added.**

In [34]:
requests.post(PUBLIC_IP + '/updateLedgerBalance', data = {
    'customerAccountId': 'bob123',
    'ledgerName': 'interestFee',
    'newBalance': 5,
    'timestamp': '2020-02-01'
})

print_current_balances()

{'applicationFee': 10,
 'interestFee': 5,
 'interestFeePaid': 0,
 'lateFees': 0,
 'principal': 1000,
 'principalPaid': 0}


**On 3/1/2020, Bob makes a payment of \\$100 and as a result, the interest fee is reduced to \\$0 and principal goes down by \\$95 to a balance of \\$905. A principal paid ledger type is added with a value of \\$95 and an interest fee paid ledger type is added with a value of \\$5.**

In [29]:
current_balances = requests.get(PUBLIC_IP + '/getCurrentLedgerBalances?customerAccountId=bob123').json()


# Payment Business Logic

payment = 100
interest_fee = current_balances['interestFee']
interest_fee_paid = 0
principal = current_balances['principal']
principal_paid = 0


if payment > interest_fee:
    payment -= interest_fee
    interest_fee_paid = interest_fee
    interest_fee = 0
else:
    interest_fee -= payment
    interest_fee_paid = payment
    payment = 0

if payment > principal:
    principal_paid = principal
    payment -= principal
    principal = 0
else:
    principal -= payment
    principal_paid = payment
    payment = 0

# Update ledger balances upon payment

requests.post(PUBLIC_IP + '/updateLedgerBalance', data = {
    'customerAccountId': 'bob123',
    'ledgerName': 'interestFee',
    'newBalance': interest_fee,
    'timestamp': '2020-03-01'
})

requests.post(PUBLIC_IP + '/updateLedgerBalance', data = {
    'customerAccountId': 'bob123',
    'ledgerName': 'interestFeePaid',
    'newBalance': interest_fee_paid,
    'timestamp': '2020-03-01'
})

requests.post(PUBLIC_IP + '/updateLedgerBalance', data = {
    'customerAccountId': 'bob123',
    'ledgerName': 'principal',
    'newBalance': principal,
    'timestamp': '2020-03-01'
})

requests.post(PUBLIC_IP + '/updateLedgerBalance', data = {
    'customerAccountId': 'bob123',
    'ledgerName': 'principalPaid',
    'newBalance': principal_paid,
    'timestamp': '2020-03-01'
})


<Response [200]>

**Bob's most updated balance should reflect the new changes.**

In [36]:
print_current_balances()

{'applicationFee': 10,
 'interestFee': 0,
 'interestFeePaid': 5,
 'lateFees': 0,
 'principal': 905,
 'principalPaid': 95}


**On 4/1/2020 the interest fee amount again goes up by \\$5.**

In [37]:
requests.post(PUBLIC_IP + '/updateLedgerBalance', data = {
    'customerAccountId': 'bob123',
    'ledgerName': 'interestFee',
    'newBalance': 5,
    'timestamp': '2020-04-01'
})

print_current_balances()

{'applicationFee': 10,
 'interestFee': 5,
 'interestFeePaid': 5,
 'lateFees': 0,
 'principal': 905,
 'principalPaid': 95}


**On 4/5/2020 the bank decides that the customer is late on their monthly payment and add \\$40 to a new ledger type late fees**

In [38]:
requests.post(PUBLIC_IP + '/updateLedgerBalance', data = {
    'customerAccountId': 'bob123',
    'ledgerName': 'lateFees',
    'newBalance': 40,
    'timestamp': '2020-04-05'
})

print_current_balances()

{'applicationFee': 10,
 'interestFee': 5,
 'interestFeePaid': 5,
 'lateFees': 40,
 'principal': 905,
 'principalPaid': 95}


**There have been quite a few transactions on Bob's account! Let's check out his <span style="color:red">full account history to see the ledger changes over time</span>. The balances and timestamps should be in <span style="color:red">chronological order</span> and match up to the previous updates.**

In [39]:
r = requests.get(PUBLIC_IP + '/getCustomerAccountData?customerAccountId=bob123')
pprint.pprint(r.json())

{'_id': '5ed44fd829979d0e93750d68',
 'customerAccountId': 'bob123',
 'firstName': 'Bob',
 'lastName': 'Bobberman',
 'ledgers': {'applicationFee': [{'balance': 0,
                                 'timestamp': '1970-01-01T00:00:00.000Z'},
                                {'balance': 10,
                                 'timestamp': '2020-01-01T00:00:00.000Z'}],
             'interestFee': [{'balance': 0,
                              'timestamp': '1970-01-01T00:00:00.000Z'},
                             {'balance': 5,
                              'timestamp': '2020-02-01T00:00:00.000Z'},
                             {'balance': 0,
                              'timestamp': '2020-03-01T00:00:00.000Z'},
                             {'balance': 5,
                              'timestamp': '2020-04-01T00:00:00.000Z'}],
             'interestFeePaid': [{'balance': 0,
                                  'timestamp': '1970-01-01T00:00:00.000Z'},
                                 {'balance': 5,
  

**Let's say we want the <span style="color:red">balance of a particular ledger (interest fee) on a specific date (February 1st, 2020)</span>.**

In [40]:
r = requests.get(PUBLIC_IP + '/getLedgerBalanceByDate?customerAccountId=bob123&ledgerName=interestFee&timestamp=2020-02-01')
pprint.pprint(r.json())

{'interestFee': 5, 'timestamp': '2020-02-01T00:00:00.000Z'}


**If there is no exact matches to the date, the API will return the most recent record as of that date.**

In [44]:
r = requests.get(PUBLIC_IP + '/getLedgerBalanceByDate?customerAccountId=bob123&ledgerName=interestFee&timestamp=2020-05-05')
pprint.pprint(r.json())

{'interestFee': 5, 'timestamp': '2020-04-01T00:00:00.000Z'}


**Finally, if we ever want to <span style="color:red">add a NEW ledger type</span>, we can simply specify our new ledger name and corresponding balance using the same /updateLedgerBalance endpoint! This makes the system very easy to expand.**

In [43]:
requests.post(PUBLIC_IP + '/updateLedgerBalance', data = {
    'customerAccountId': 'bob123',
    'ledgerName': 'securityDeposit',
    'newBalance': 5005.50,
    'timestamp': '2020-05-31'
})

print_current_balances()

{'applicationFee': 10,
 'interestFee': 5,
 'interestFeePaid': 5,
 'lateFees': 40,
 'principal': 905,
 'principalPaid': 95,
 'securityDeposit': 5005}
